<a href="https://colab.research.google.com/github/repitta/CienciaDeDadosEducacionais/blob/master/pre-processamento%20notasListas%20e%20notaSigaa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importando as Bibliotecas**

In [0]:
import pandas as pd

**Lendo o arquivo CSV lopUsuarioNota**

In [0]:
# Ler o arquivo CSV 
lopNotasLista  = pd.read_csv("https://github.com/repitta/CienciaDeDadosEducacionais/raw/master/arquivosCSV/notasListas.csv")

**Estrutura do aquivo**

In [3]:
print(lopNotasLista.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7495 entries, 0 to 7494
Data columns (total 15 columns):
Unnamed: 0              7495 non-null int64
titulo                  7495 non-null object
idAluno                 7495 non-null object
anoSubmissao            7495 non-null float64
SomaLista               7495 non-null float64
questoesFeitas          7495 non-null int64
quantidadeTotalLista    7495 non-null float64
NotaLista               7438 non-null float64
anoLop                  7495 non-null float64
turma                   7495 non-null object
matricula               7495 non-null int64
nome                    7495 non-null object
notaSigaa               7495 non-null object
situacao                7495 non-null object
anoSigaa                7495 non-null float64
dtypes: float64(6), int64(3), object(6)
memory usage: 878.4+ KB
None


**Pré-processamento**

In [0]:
##Tabela lopNotasLista
#excluir a coluna Unnamed
lopNotasLista = lopNotasLista.drop("Unnamed: 0", axis=1)
#dividir a coluna NotasLista por 10 , pq elas estão em porcentagem
lopNotasLista["NotaLista"] = lopNotasLista["NotaLista"] /10
#colocar letra maiuscula na coluna nome
lopNotasLista["nome"] = lopNotasLista["nome"].str.upper()
lopNotasLista = lopNotasLista.loc[~lopNotasLista["titulo"].str.contains(r"^0"),:]

In [5]:
lopNotasLista["titulo"].unique()

array(['Lista 1.0 - Expressões aritméticas',
       'Lista 1.2 - Funções aritméticas',
       'Lista de Exercícios 1 - Expressões e variáveis',
       'Lista de Exercícios 2 - Estruturas Condicionais',
       'Lista de Exercícios 3 - Laço Condicional',
       'Lista de Exercícios 4 - Laço contado',
       'Lista de Exercícios 4.1 - Problemas tradicionais com repetição contada',
       'Lista de Exercícios 5 - Vetores',
       'Lista de Laboratório 1 - Expressões Aritméticas e Variáveis',
       'Lista de Laboratório 2 - Expressões Lógicas e Estruturas de Seleção',
       'Lista de Laboratório 3 -  Estruturas de Repetição Condicional',
       'Lista de Laboratório 4 - Repetição Contada',
       'Lista de Laboratório 5 - Vetores', 'Lista de Revisão Unidade 1',
       'Prova 1 - Turma 09 - 2017.1', 'Prova 1 - Turma 3A - 2018.2',
       'Prova 1 - Turma 3B - 2018.2', 'Prova 1 - Turma 3C - 2018.2',
       'Prova 1 - Turma 3D - 2018.2', 'Prova 1 - Turmas 2A e 2B - 2018.1',
       'Prova 1 - 

**Funcao para formar a tabela de notas de todas as listas de exercicios  por ano e por turma**

In [0]:
def tabelaNotasListaAnoTurma(ano,turma):
  colunas    = ["matricula","nome"]
  tabelaListas = pd.DataFrame(columns=colunas)
  listas = ["Exercícios 5", "Exercícios 4.1", "Exercícios 4 - ", "Exercícios 3", "Exercícios 2","Exercícios 1",
            "Laboratório 5","Laboratório 4", "Laboratório 3", "Laboratório 2","Laboratório 1"
           ]
  for l in listas:
    notasTurma = lopNotasLista.loc[(lopNotasLista["anoSigaa"] == ano) & (lopNotasLista["turma"].str.contains(turma, regex=False)) &
                 (lopNotasLista["titulo"].str.contains(l, regex=False)),["matricula","nome","NotaLista","questoesFeitas"]]
    notasTurma = notasTurma.sort_values("nome")
    tabelaListas = pd.merge(notasTurma,tabelaListas,on=['matricula','nome'], how="outer") 
    
  tabelaListas.columns = ['matricula', 'nome',"lab1","qLab1", "lab2","qLab2","lab3","qLab3","lab4","qLab4","lab5","qLab5","exe1","qexe1","exe2","qexe2",
  "exe3","qexe3","exe4","qexe4","exe4.1","qexe4.1","exe5","qexe5"]
  tabelaListas = tabelaListas.drop_duplicates()
  tabelaListas = tabelaListas.sort_values('nome')
  return tabelaListas

In [0]:
#retirar o semestre de 2019.2 pq os alunos não terminaram todas as listas
lopNotasLista = lopNotasLista.loc[lopNotasLista["anoSigaa"]!= 2019.2,:]

***Criar um arquivo csv com as notas das listas por turma***

In [0]:
anos = lopNotasLista["anoSigaa"].unique()
turmas = lopNotasLista["turma"].unique()
tabelaTotal = pd.DataFrame()
for a in anos:
  for t in turmas:
    tabelaLista = tabelaNotasListaAnoTurma(a,t)
    tabelaTotal = pd.concat([tabelaTotal,tabelaLista], ignore_index =True)
tabelaTotal

In [0]:
#juntas as tabelas com ficar as notas das listas e a nota final no sigaa
notaSigaa = lopNotasLista[["anoSigaa","matricula","nome","turma", "notaSigaa","situacao"]]
mergeListasNotaSigaa = pd.merge(notaSigaa,tabelaTotal,on=['matricula','nome'], how="outer") 
mergeListasNotaSigaa = mergeListasNotaSigaa.drop_duplicates()
mergeListasNotaSigaa = mergeListasNotaSigaa.sort_values(by=["anoSigaa","turma","nome"])

In [0]:
#Exportar o arquivo csv
mergeListasNotaSigaa.to_csv(r"ListasNotaSigaa.csv")
